In [ ]:
import requests
from urllib.parse import urlencode
from io import BytesIO
from zipfile import ZipFile
import os

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/6EQp8FpoaYkTwQ'

# Формируем URL для получения href
final_url = base_url + urlencode({'public_key': public_key})
response = requests.get(final_url)
response.raise_for_status()
download_url = response.json()['href']

# Скачиваем ZIP с прогрессом
print("Начинаем скачивать датасет...")
r = requests.get(download_url, stream=True)
r.raise_for_status()

total_size = int(r.headers.get('content-length', 0))
downloaded = 0
zip_bytes = BytesIO()

for chunk in r.iter_content(chunk_size=8192):
    if not chunk:
        continue
    zip_bytes.write(chunk)
    downloaded += len(chunk)
    mb_downloaded = downloaded / (1024 * 1024)
    print(f'\rСкачано {mb_downloaded:.2f} MiB', end='')

print()  # перенос на новую строку после прогресса
zip_bytes.seek(0)
print("Датасет скачан.")

# Путь, куда хотим распаковать датасет
dist_path = '/home/jupyter/datasphere/project/fire_smoke_dataset'
os.makedirs(dist_path, exist_ok=True)
print(f"Распаковываем датасет в {dist_path}")

# Распаковываем вручную, убирая первые два уровня папок
with ZipFile(zip_bytes) as z:
    for member in z.infolist():
        parts = member.filename.split('/')
        # Пропускаем всё, что не «глубже» двух уровней
        if len(parts) <= 2:
            continue
        # Собираем путь, начиная с третьего элемента
        relative_path = os.path.join(*parts[2:])
        target_path = os.path.join(dist_path, relative_path)

        if member.is_dir():
            os.makedirs(target_path, exist_ok=True)
        else:
            os.makedirs(os.path.dirname(target_path), exist_ok=True)
            with z.open(member) as src, open(target_path, 'wb') as dst:
                dst.write(src.read())

print(f'Готово! Данные разложены в {dist_path}')


Начинаем скачивать датасет...
Скачано 1341.85 MiB
Датасет скачан.
Распаковываем датасет в /home/jupyter/datasphere/project/fire_smoke_dataset
Готово! Данные разложены в /home/jupyter/datasphere/project/fire_smoke_dataset


In [2]:
# 0: smoke
# 1: fire
import xml.etree.ElementTree as ET
import os

pathDataset = '/home/jupyter/datasphere/project/fire_smoke_dataset'
os.makedirs(f'{pathDataset}/labels', exist_ok=True)

tree = ET.parse(f"{pathDataset}/annotations.xml")
root = tree.getroot()

nameToNumber = {'smoke': 0, 'fire': 1}

cnt = 0
for child in root:
    if cnt < 2:
        cnt += 1
        continue

    name = child.attrib.get('name')
    data = ''
    for subChild in child:
        label = subChild.attrib.get('label')
        x1 = subChild.attrib.get('xtl')
        y1 = subChild.attrib.get('ytl')
        x2 = subChild.attrib.get('xbr')
        y2 = subChild.attrib.get('ybr')
        bbox = [x1, y1, x2, y2]

        data += str(nameToNumber[label])
        for cord in bbox:
            if cord != 0:
                data = data + ' ' + str(int(float(cord))/640)
        data += '\n'
    cnt += 1

    with open(f'{pathDataset}/labels/{name.replace(".jpg", ".txt")}', 'w') as file:
        file.write(data)

In [3]:
import os
import shutil
pathDataset = '/home/jupyter/datasphere/project/fire_smoke_dataset'

allFile = [f for f in os.listdir(f'{pathDataset}/labels') if f.endswith('.txt')]
cntData = len(allFile)

# разбиаение датасета
n_test  = int(cntData * 0.05)
n_val   = int(cntData * 0.10)
n_train = cntData - n_test - n_val
    
# Разбиваем индексы
indices = list(range(cntData))
test_idx  = indices[:n_test]
val_idx   = indices[n_test:n_test+n_val]
train_idx = indices[n_test+n_val:]

splits = {
    'test':  test_idx,
    'val':   val_idx,
    'train': train_idx
}

# Создаём папки назначения
for kind in ['labels', 'images']:
    for split in ['train','val','test']:
        os.makedirs(f'{pathDataset}/{kind}/{split}', exist_ok=True)

# Перемещаем файлы по сплитам
for split, idxs in splits.items():
    for i in idxs:
        name = os.path.splitext(allFile[i])[0]  # имя без .txt
        src_lbl = f'{pathDataset}/labels/{name}.txt'
        dst_lbl = f'{pathDataset}/labels/{split}/{name}.txt'
        src_img = f'{pathDataset}/images/{name}.jpg'
        dst_img = f'{pathDataset}/images/{split}/{name}.jpg'

        # Move (shutil.move сам создаёт папки, но на всякий случай с)
        shutil.move(src_lbl, dst_lbl)
        shutil.move(src_img, dst_img)

print("Данные разбиты на train/val/test")

print("Проверка разбивки")
for kind in ['labels', 'images']:
    for split in ['train','val','test']:
        a = len(os.listdir(f'{pathDataset}/{kind}/{split}'))
        print(a)
        
os.remove(f'{pathDataset}/annotations.xml')


Данные разбиты на train/val/test
Проверка разбивки
5820
684
342
5820
684
342


In [1]:
#!:bash

set -e
cp -r /home/jupyter/datasphere/project/fire_smoke_dataset/* /home/jupyter/mnt/datasets/fire_smoke_dataset

cp: cannot create regular file '/home/jupyter/mnt/datasets/fire_smoke_dataset/data.yaml': Read-only file system


Exception: Process exited with code 1